# Cross Validation
## Introduction

In an earlier mission, we learned about train/test validation, a simple technique for testing a machine learning model's accuracy on new data that the model wasn't trained on. In this mission, we'll focus on more robust techniques.<br>

To start, we'll focus on the **holdout validation** technique, which involves:

* splitting the full dataset into 2 partitions:
  * a training set
  * a test set
* training the model on the training set,
* using the trained model to predict labels on the test set,
* computing an error metric to understand the model's effectiveness,
* switch the training and test sets and repeat,
* average the errors.

In holdout validation, we usually use a 50/50 split instead of the 75/25 split from train/test validation. This way, we remove number of observations as a potential source of variation in our model performance.

![](https://s3.amazonaws.com/dq-content/holdout_validation.png)

Let's start by splitting the data set into 2 nearly equivalent halves.

* Use the `numpy.random.permutation()` function to shuffle the ordering of the rows in `dc_listings`.
* Select the first 1862 rows and assign to `split_one`.
* Select the remaining 1861 rows and assign to `split_two`.

In [1]:
import numpy as np
import pandas as pd

dc_listings = pd.read_csv("data/dc_airbnb.csv")
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

dc_listings = dc_listings.reindex(np.random.permutation(dc_listings.index))
split_one = dc_listings.iloc[:1862]
split_two = dc_listings.iloc[1862:]

## Holdout Validation

Now that we've split our data set into 2 dataframes, let's:

* train a k-nearest neighbors model on the first half,
* test this model on the second half,
* train a k-nearest neighbors model on the second half,
* test this model on the first half.

* Train a k-nearest neighbors model using the default algorithm (auto) and the default number of neighbors (5) that:
  * Uses the accommodates column from train_one for training and
  * Tests it on test_one.
* Assign the resulting RMSE value to iteration_one_rmse.
* Train a k-nearest neighbors model using the default algorithm (auto) and the default number of neighbors (5) that:
  * Uses the accommodates column from train_two for training and
  * Tests it on test_two.
* Assign the resulting RMSE value to iteration_two_rmse.
* Use numpy.mean() to calculate the average of the 2 RMSE values and assign to `avg_rmse`.

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [16]:
train_one, test_one = split_one, split_two
features = ['accommodates']

knn = KNeighborsRegressor(n_neighbors=5, algorithm='auto')
knn.fit(train_one[features], train_one['price'])

iteration_one_rmse = np.sqrt(mean_squared_error(test_one['price'],
                                               knn.predict(test_one[features])))
iteration_one_rmse

122.72875033169522

In [17]:
train_two, test_two = split_two, split_one

knn = KNeighborsRegressor(n_neighbors=5, algorithm='auto')
knn.fit(train_two[features], train_two['price'])

iteration_two_rmse = np.sqrt(mean_squared_error(test_two['price'],
                                               knn.predict(test_two[features])))
iteration_two_rmse

131.62692412116255

In [18]:
avg_rmse = np.mean([iteration_one_rmse, iteration_two_rmse])
avg_rmse

127.17783722642889

## K-Fold Cross Validation

If we average the two RMSE values from the last step, we get an RMSE value of approximately **127.18**. Holdout validation is actually a specific example of a larger class of validation techniques called **k-fold cross-validation**. While holdout validation is better than train/test validation because the model isn't repeatedly biased towards a specific subset of the data, both models that are trained only use half the available data. K-fold cross validation, on the other hand, takes advantage of a larger proportion of the data during training while still rotating through different subsets of the data to avoid the issues of train/test validation.<br>

Here's the algorithm from k-fold cross validation:

* splitting the full dataset into `k` equal length partitions,
  * selecting `k-1` partitions as the training set and
  * selecting the remaining partition as the test set
* training the model on the training set,
* using the trained model to predict labels on the test fold,
* computing the test fold's error metric,
* repeating all of the above steps `k-1` times, until each partition has been used as the test set for an iteration,
* calculating the mean of the `k` error values.

Holdout validation is essentially a version of k-fold cross validation when `k` is equal to `2`. Generally, `5` or `10` folds is used for k-fold cross-validation. Here's a diagram describing each iteration of 5-fold cross validation:

![](https://s3.amazonaws.com/dq-content/kfold_cross_validation.png)

As you increase the number the folds, the number of observations in each fold decreases and the variance of the fold-by-fold errors increases. Let's start by manually partitioning the data set into 5 folds. Instead of splitting into 5 dataframes, let's add a column that specifies which fold the row belongs to. This way, we can easily select


* Add a new column to `dc_listings` named `fold` that contains the `fold` number each row belongs to:
  * Fold `1` should have rows from index `0` to 744, including both of those rows.
  * Fold `2` should have rows from index `744` to `1488, including both of those rows.
  * Fold `3` should have rows from index `1488` to `2232`, including both of those rows.
  * Fold `4` should have rows from index `2232` to `2976`, including both of those rows.
  * Fold `5` should have rows from index `2976` to `3723`, including both of these rows.
* Display the unique value counts for the `fold` column to confirm that each fold has roughly the same number of elements.

In [20]:
dc_listings.set_value(dc_listings.index[0:744], "fold", 1)
dc_listings.set_value(dc_listings.index[744:1488], "fold", 2)
dc_listings.set_value(dc_listings.index[1488:2232], "fold", 3)
dc_listings.set_value(dc_listings.index[2232:2976], "fold", 4)
dc_listings.set_value(dc_listings.index[2976:3723], "fold", 5)

dc_listings['fold'].value_counts()

5.0    747
4.0    744
3.0    744
2.0    744
1.0    744
Name: fold, dtype: int64

## First iteration

Let's start by performing the first iteration of k-fold cross validation on a simple, univariate model.

* Train a k-nearest neighbors model using the `accommodates` column as the sole feature from folds `2` to `5` as the training set.
* Use the model to make predictions on the test set (`accommodates` column from fold `1`) and assign the predicted labels to `labels`.
* Calculate the RMSE value by comparing the `price` column with the predicted labels.
* Assign the RMSE value to `iteration_one_rmse`.

In [32]:
train_set = dc_listings[dc_listings['fold'] != 1]
test_set = dc_listings[dc_listings['fold'] == 1]

knn = KNeighborsRegressor()
knn.fit(train_set['accommodates'].values.reshape(-1,1), train_set['price'])

labels = knn.predict(test_set['accommodates'].values.reshape(-1,1))
iteration_one_rmse = np.sqrt(mean_squared_error(test_set['price'], labels))

iteration_one_rmse

131.39679747656245

## Function for training models

From the first iteration, we achieved an RMSE value of **131.4**. Let's calculate the RMSE values for the remaining iterations. To make the iteration process easier, let's wrap the code we wrote in the previous screen in a function.

* Write a function named `train_and_validate` that takes in a dataframe as the first parameter (`df`) and a list of fold values (`1` to `5` in our case) as the second parameter (`folds`). This function should:
  * Train `n` models (where `n` is number of folds) and perform k-fold cross validation (using `n` folds). Use the default `k` value for the `KNeighborsRegressor` class.
  * Return a list of RMSE values, where the first element is the RMSE for when fold `1` was the test set, the second element is the RMSE for when fold `2` was the test set, and so on.
* Use the train_and_validate function to return the list of RMSE values for the `dc_listings` Dataframe and assign to `rmses`.
* Calculate the mean of these values and assign to `avg_rmse`.
* Display both `rmses` and `avg_rmse`.

In [36]:
def train_and_validate(df, folds):
    
    rmses = []
    
    for fold in folds:
        
        train_set = df[df['fold'] != fold]
        test_set = df[df['fold'] == fold]
       
        knn = KNeighborsRegressor()
        knn.fit(train_set['accommodates'].values.reshape(-1,1), train_set['price'])

        labels = knn.predict(test_set['accommodates'].values.reshape(-1,1))
        iteration_rmse = np.sqrt(mean_squared_error(test_set['price'], labels))
        
        rmses.append(iteration_rmse)
    
    return rmses


rmses = train_and_validate(dc_listings, range(1,6))
avg_rmse = np.mean(rmses)

print(rmses)
print(avg_rmse)

[131.39679747656245, 132.07741945988644, 181.77725261949712, 103.99703159535449, 130.67234702020832]
135.984169634


## Performing K-Fold Cross Validation Using Scikit-Learn

While the average RMSE value was approximately **135.4**, the RMSE values ranged from **104.0** all the way to **181.8**. This large amount of variability between the RMSE values means that we're either using a poor model or a poor evaluation criteria (or a bit of both!). By implementing your own k-fold cross-validation function, you hopefully acquired a good understanding of the inner workings of the technique. The function we wrote, however, has many limitations. If we want to now change the number of folds we want to use, we need to make the function more general so it can also handle randomizing the ordering of the rows in the dataframe and splitting into folds.<br>

In machine learning, we're interested in building a good model and accurately understand how well it will perform. To build a better k-nearest neighbors model, we can change the features it uses or tweak the number of neighbors (a hyperparameter). To accurately understand a model's performance, we can perform k-fold cross validation and select the proper number of folds. We've learned how scikit-learn makes it easy for us to quickly experiment with these different knobs when it comes to building a better model. Let's now dive into how we can use scikit-learn to handle cross-validation as well.<br>

First, we instantiate an instance of the [KFold class](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) from `sklearn.model_selection`:

```python
from sklearn.model_selection import KFold
kf = KFold(n_splits, shuffle=False, random_state=None)
```

where:

* `n_splits` is the number of folds you want to use,
* `shuffle` is used to toggle shuffling of the ordering of the observations in the dataset,
* `random_state` is used to specify the random seed value if `shuffle` is set to `True`.

You'll notice here that no parameters depend on the data set at all. This is because the KFold class returns an iterator object which we use in conjunction with the [cross_val_score() function](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), also from `sklearn.model_selection`. Together, these 2 functions allow us to compactly train and test using k-fold cross validation:

Here are the relevant parameters for the `cross_val_score` function:

```python
from sklearn.model_selection import cross_val_score
cross_val_score(estimator, X, Y, scoring=None, cv=None)
```

where:

* `estimator` is a sklearn model that implements the `fit` method (e.g. instance of KNeighborsRegressor),
* `X` is the list or 2D array containing the features you want to train on,
* `y` is a list containing the values you want to predict (target column),
* `scoring` is a string describing the scoring criteria (list of accepted values here).
* `cv` describes the number of folds. Here are some examples of accepted values:
  * an instance of the `KFold` class,
  * an integer representing the number of folds.

Depending on the scoring criteria you specify, either a single total value is returned one value for each fold. Here's the general workflow for performing k-fold cross-validation using the classes we just described:

* instantiate the scikit-learn model class you want to fit,
* instantiate the `KFold` class and using the parameters to specify the k-fold cross-validation attributes you want,
* use the `cross_val_score()` function to return the scoring metric you're interested in.


### `scoring` parameter options (`cross_val_score` function)
['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'mutual_info_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'v_measure_score']

* Create a new instance of the `KFold` class with the following properties:
  * `5` folds,
  * shuffle set to `True`,
  * random seed set to `1` (so we can answer check using the same seed),
  * assigned to the variable `kf`.

* Create a new instance of the `KNeighborsRegressor` class and assign to `knn`.
* Use the `cross_val_score()` function to perform k-fold cross-validation:
  * using the KNeighborsRegressor instance `knn`,
  * using the `accommodates` column for training,
  * using the `price` column as the target column,
  * returning an array of MSE values (one value for each fold).
* Assign the resulting list of MSE values to `mses`. Then, take the absolute value followed by the square root of each mse value. Then, calculate the average of the resulting RMSE values and assign to `avg_rmse`.

In [37]:
from sklearn.model_selection import cross_val_score, KFold

In [45]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
knn = KNeighborsRegressor()
mses = cross_val_score(estimator=knn,
                      X = dc_listings['accommodates'].values.reshape(-1,1),
                      y = dc_listings['price'],
                      cv = kf,
                      scoring = 'neg_mean_squared_error')

avg_rmse = pd.Series(mses).apply(lambda x: np.sqrt(abs(x))).mean()

In [42]:
avg_rmse

130.1106896801877

## Exploring Different K Values

Choosing the right `k` value when performing k-fold cross validation is **more of an art and less of a science**. As we discussed earlier in the mission, a k value of `2` is really just holdout validation. On the other end, setting `k` equal to `n` (the number of observations in the data set) is known as **leave-one-out cross validation**, or **LOOCV** for short. Through lots of trial and error, data scientists have converged on `10` as the `standard k value`. <br>

In the following code block, we display the results of varying `k` from `3` to `23`. For each `k` value, we calculate and display the average RMSE value across all of the folds and the standard deviation of the RMSE values. Across the many different `k` values, it seems like the average RMSE value is around `128`. You'll notice that the standard deviation of the RMSE increases from approximately `1.1` to `37.3` **as we increase the number the folds**.

In [43]:
from sklearn.model_selection import cross_val_score, KFold

num_folds = [3, 5, 7, 9, 10, 11, 13, 15, 17, 19, 21, 23]

for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates"]], dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print(str(fold), "folds: ", "avg RMSE: ", str(avg_rmse), "std RMSE: ", str(std_rmse))

3 folds:  avg RMSE:  131.499007862 std RMSE:  8.31393074701
5 folds:  avg RMSE:  130.11068968 std RMSE:  22.2246985957
7 folds:  avg RMSE:  127.076513904 std RMSE:  22.5677559934
9 folds:  avg RMSE:  129.720042328 std RMSE:  16.9960277409
10 folds:  avg RMSE:  127.560345705 std RMSE:  26.8820755914
11 folds:  avg RMSE:  125.581172838 std RMSE:  32.904083491
13 folds:  avg RMSE:  133.701500957 std RMSE:  29.8304567392
15 folds:  avg RMSE:  127.59573416 std RMSE:  38.8814545015
17 folds:  avg RMSE:  124.055485979 std RMSE:  40.878770978
19 folds:  avg RMSE:  127.390946734 std RMSE:  40.8205853927
21 folds:  avg RMSE:  128.736078565 std RMSE:  44.2899275721
23 folds:  avg RMSE:  129.13596586 std RMSE:  42.993633634


## Bias-Variance Tradeoff

So far, we've been working under the assumption that a lower RMSE always means that a model is more accurate. This isn't the complete picture, unfortunately. A model has two sources of error, **bias** and **variance**.<br>

Bias describes error that results in bad assumptions about the learning algorithm. For example, assuming that only one feature, like a car's weight, relates to a car's fuel efficiency will lead you to fit a simple, univariate regression model that will result in high bias. The error rate will be high since a car's fuel efficiency is affected by many other factors besides just its weight.

Variance describes error that occurs because of the variability of a model's predicted values. If we were given a dataset with 1000 features on each car and used every single feature to train an incredibly complicated multivariate regression model, we will have low bias but high variance. In an ideal world, we want low bias and low variance but in reality, there's always a tradeoff.

The standard deviation of the RMSE values can be a proxy for a model's **variance** while the average RMSE is a proxy for a model's **bias**. Bias and variance are the 2 observable sources of error in a model that we can indirectly control.

![](https://s3.amazonaws.com/dq-content/bias_variance.png)

While k-nearest negihbors can make predictions, it isn't a mathematical model. A mathematical model is usually an equation that can exist without the original data, which isn't true with k-nearest neighbors. In the next two courses, we'll learn about a mathematical model called linear regression. We'll explore the bias-variance tradeoff in greater depth in these next 2 courses because of its importance when working with mathematical models in particular.